# 🐺 TEST 2: Crash Bounce - Does It Work?
## Claim: 69% win rate buying -15%+ drops with RSI < 40

**Goal:** Validate with real historical data

**Decision Rule:**
- If win rate > 60% → BUILD IT
- If not → DELETE THIS NOTEBOOK

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Ready to test crash bounce")

---
## Backtest Function

In [ ]:
def calculate_rsi(prices, period=14):
    """Calculate RSI"""
    deltas = np.diff(prices)
    seed = deltas[:period+1]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down if down != 0 else 0
    rsi = np.zeros_like(prices)
    rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period

        rs = up / down if down != 0 else 0
        rsi[i] = 100. - 100. / (1. + rs)

    return rsi


def test_crash_bounce(ticker, crash_threshold=-15, rsi_threshold=40, hold_days=3, months=12):
    """Test crash bounce strategy on a ticker"""
    
    stock = yf.Ticker(ticker)
    hist = stock.history(period=f'{months}mo')
    
    if len(hist) < 30:
        return None
    
    # Calculate daily returns and RSI
    hist['Return'] = hist['Close'].pct_change() * 100
    hist['RSI'] = calculate_rsi(hist['Close'].values)
    
    # Find crash days
    crashes = []
    
    for i in range(20, len(hist) - hold_days):  # Need space for RSI calc and hold period
        daily_return = hist['Return'].iloc[i]
        rsi = hist['RSI'].iloc[i]
        
        # Crash condition: down -15%+ AND RSI < 40
        if daily_return <= crash_threshold and rsi < rsi_threshold:
            
            entry_price = hist['Close'].iloc[i]
            exit_price = hist['Close'].iloc[i + hold_days]
            
            trade_return = ((exit_price - entry_price) / entry_price) * 100
            
            crashes.append({
                'date': hist.index[i].strftime('%Y-%m-%d'),
                'crash_pct': daily_return,
                'rsi': rsi,
                'entry': entry_price,
                'exit': exit_price,
                'return': trade_return,
                'win': trade_return > 0
            })
    
    if len(crashes) == 0:
        print(f"\n❌ {ticker}: No crashes found in last {months} months")
        return None
    
    # Calculate stats
    df = pd.DataFrame(crashes)
    win_rate = (df['win'].sum() / len(df)) * 100
    avg_return = df['return'].mean()
    avg_win = df[df['win']]['return'].mean() if any(df['win']) else 0
    avg_loss = df[~df['win']]['return'].mean() if any(~df['win']) else 0
    
    # Print results
    print(f"\n{'='*70}")
    print(f"📊 {ticker} - CRASH BOUNCE TEST ({months} months)")
    print(f"{'='*70}")
    
    print(f"\n📉 Crash Setup: Down {crash_threshold}%+, RSI < {rsi_threshold}")
    print(f"⏱️  Hold Period: {hold_days} days")
    print(f"📊 Sample Size: {len(df)} trades")
    
    print(f"\n📈 RESULTS:")
    print(f"   Win Rate: {win_rate:.1f}% ({df['win'].sum()}/{len(df)} trades)")
    print(f"   Average Return: {avg_return:+.2f}%")
    print(f"   Average Win: {avg_win:+.2f}%")
    print(f"   Average Loss: {avg_loss:+.2f}%")
    
    # Show last 5 trades
    print(f"\n📅 LAST 5 CRASH BOUNCES:")
    for _, trade in df.tail(5).iterrows():
        emoji = "✅" if trade['win'] else "❌"
        print(f"   {emoji} {trade['date']}: Crash {trade['crash_pct']:.1f}%, RSI {trade['rsi']:.0f} → {trade['return']:+.2f}% ({hold_days}d)")
    
    # Verdict
    print(f"\n{'='*70}")
    if win_rate >= 60:
        print(f"✅ EDGE CONFIRMED: {win_rate:.1f}% win rate")
    else:
        print(f"❌ NO EDGE: {win_rate:.1f}% win rate (need 60%+)")
    print(f"{'='*70}")
    
    return {
        'ticker': ticker,
        'trades': len(df),
        'win_rate': win_rate,
        'avg_return': avg_return,
        'edge_exists': win_rate >= 60
    }

# Test WULF (claimed 69% win rate)
wulf_result = test_crash_bounce('WULF', crash_threshold=-15, rsi_threshold=40, hold_days=3, months=12)

---
## Test Multiple Tickers

In [ ]:
# Test across multiple volatile tickers
test_tickers = ['WULF', 'CIFR', 'IREN', 'APLD', 'RCAT', 'SOUN', 'BBAI', 'SMR']

results = []

for ticker in test_tickers:
    result = test_crash_bounce(ticker, crash_threshold=-15, rsi_threshold=40, hold_days=3, months=12)
    if result:
        results.append(result)

# Summary
if len(results) > 0:
    print("\n" + "="*70)
    print("📊 CRASH BOUNCE SUMMARY")
    print("="*70)
    
    df = pd.DataFrame(results)
    df = df.sort_values('win_rate', ascending=False)
    
    print(f"\n{'Ticker':<8} {'Trades':<8} {'Win Rate':<12} {'Avg Return':<12} {'Edge?':<8}")
    print("-" * 70)
    
    for _, row in df.iterrows():
        edge_symbol = "✅" if row['edge_exists'] else "❌"
        print(f"{row['ticker']:<8} {row['trades']:<8.0f} {row['win_rate']:5.1f}%      {row['avg_return']:+6.2f}%      {edge_symbol:<8}")
    
    # Overall assessment
    edge_count = sum(df['edge_exists'])
    total_count = len(df)
    overall_win_rate = df['win_rate'].mean()
    
    print("\n" + "="*70)
    if edge_count >= 3 or overall_win_rate >= 60:
        print(f"✅ CRASH BOUNCE EDGE IS REAL")
        print(f"   Average Win Rate: {overall_win_rate:.1f}%")
        print(f"   Tickers with Edge: {edge_count}/{total_count}")
        print(f"\n💰 ACTION: Build crash_bounce_scanner.py")
        print(f"   - Run every morning")
        print(f"   - Buy crashes that meet criteria")
        print(f"   - Target +8-10%, Stop -5%")
    else:
        print(f"❌ CRASH BOUNCE EDGE IS WEAK")
        print(f"   Average Win Rate: {overall_win_rate:.1f}%")
        print(f"\n🗑️ ACTION: DELETE this notebook. Don't build the tool.")
    print("="*70)

---
## 🎯 DECISION POINT

**Look at the results:**

- ✅ **Win rate > 60%** → Build crash_bounce_scanner.py
- ❌ **Win rate < 60%** → DELETE this notebook

**The data decides. Not hopes.**